In [1]:
from __future__ import print_function
%matplotlib inline
import matplotlib.pylab as plt
import sys, os, glob
import numpy as np
import subprocess

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

from IPython.display import HTML
import xml.etree.ElementTree as ET
try:
    tree = ET.parse(os.environ['HADOOP_CONF_DIR'] + '/yarn-site.xml')
except IOError:
    raise IOError("Can't find the yarn configuration -- is HADOOP_CONF_DIR set?")
root = tree.getroot()
yarn_web_app = root.findall("./property[name='yarn.resourcemanager.webapp.address']")[0].find('value').text
yarn_web_app_string = "If this works successfully, you can check the <a target='_blank' href='http://{yarn_web_app}'>YARN application scheduler</a> and you should see your app listed there. Clicking on the 'Application Master' link will bring up the familiar Spark Web UI. "

plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

## Set up and launch the Spark runtime

Remember from the previous notebook that we have a saved configuration in `./spark_config/` -- so all we need to do is set the `SPARK_CONF_DIR` environment variable and our default configuration will be used: 

In [2]:
# specify the configuration directory
os.environ['SPARK_CONF_DIR'] = os.path.realpath('./spark_config')

# how many cores do we have for the driver
ncores = int(os.environ.get('LSB_DJOB_NUMPROC', 1)) 

# here we set the memory we want spark to use for the driver JVM
os.environ['SPARK_DRIVER_MEMORY'] = '%dG'%(ncores*0.7)

# we have to tell spark which python executable we are using
os.environ['PYSPARK_PYTHON'] = subprocess.check_output('which python', shell=True).rstrip()

import findspark
findspark.init()

import pyspark
from pyspark import SparkConf, SparkContext

conf = SparkConf()

sc = SparkContext(master='yarn-client', conf=conf)

## Initialization

Using `DataFrame`s, our entry point into the Spark universe is the `SQLContext` or the equivalent `HiveContext`:

In [3]:
from pyspark.sql import SQLContext, HiveContext

hc = HiveContext(sc)

We'll load the data off the disk, but only for the last three months of the year. 

In [4]:
%%time
data = hc.read.parquet('/user/roskarr/twitter/2014_1*')

CPU times: user 15 ms, sys: 2 ms, total: 17 ms
Wall time: 40.7 s


`data` is now a `DataFrame` object, which is essentially a collection of `Row` objects. Each `Row` object contains data for whatever columns are defined in the `DataFrame`. Here is a critical difference between `DataFrames` and `RDD`s: each column has an associated data type. While in dealing with an `RDD` we relied on Python to convert types, here each column has a specified data type. This means that:

a) we have to be a bit more careful about what we are doing and 

b) the execution engine can optimize our calculations because the data is no longer a black box. 

### Importing libraries and types for `DataFrame` API

The next few cells are a bit lengthy and complicated so you can just treat them as black boxes for now. First, we just import some necessary libraries and classes, then a series of functions are defined and executed on the `data` to give us a `DataFrame` of hashtags. 

In [5]:
import pyspark.sql.functions as func
from pyspark.sql import Row, Window
from pyspark.sql.types import IntegerType, ArrayType, StringType, NullType, LongType, StructField, StructType, DateType, DataType, DateConverter, DatetimeConverter, TimestampType, BooleanType
import datetime

### Custom functions to extract hashtags

Change the date format to YYYY-MM-DD HH:MM:SS

In [ ]:
convert_date_string = func.udf(lambda date_string: \
                               datetime.date.strftime(datetime.datetime.strptime(date_string, \
                                                                                 '%a %b %d %H:%M:%S +0000 %Y'),\
                                                      '%Y-%m-%d %H:%M:%S'), StringType())

Convert the date string to a datetime object

In [ ]:
datetime_udf = func.udf(lambda date_string: datetime.strptime(date_string, '%a %b %d %H:%M:%S +0000 %Y'), DateType())

Extract the hashtags from a hashtag array and convert them all to lowercase

In [ ]:
hash_text_udf = func.udf(lambda row: [r.text.lower() for r in row], returnType=ArrayType(StringType()))

Convert the tweet text string to lowercase

In [ ]:
key_udf_lower = func.udf(lambda row: row.lower(), returnType=StringType())

Split the text of the tweet

In [ ]:
key_udf_split = func.udf(lambda row: row.split(), returnType=ArrayType(StringType()))

Check if any of the given keywords is present in the tweet text word list. Returns a boolean.

In [ ]:
# key_udf_key = func.udf(lambda row: key in row, returnType=BooleanType())
key_udf_key = func.udf(lambda row: len([val for val in keys if val in row]) > 0, returnType=BooleanType())

Combine the year of creation of the tweet and the day of creation to a unique date ID.

In [ ]:
new_date = (lambda col: func.dayofyear(col) + func.year(col)*1000)

### Apply the functions to the data frame

Define the number of partitions for the data

In [ ]:
Npartitions = sc.defaultParallelism*5

Create the hashtag dataframe with all the tweets with more than one hashtag, extract the hashtags and also extract the creation date and convert it to string.

In [26]:
# only keep the tweets with at least one hashtag
hashtag_df = (data.select('created_at', 'entities.hashtags', 'text')
                .filter(func.size('hashtags') > 1)
                .withColumn('hash_text', hash_text_udf('hashtags'))
                .withColumn('created_at', convert_date_string('created_at'))
                .repartition(Npartitions))
hashtag_df.cache()

DataFrame[created_at: string, hashtags: array<struct<indices:array<bigint>,text:string>>, text: string, hash_text: array<string>]

In [27]:
hashtag_df.show(10)

+-------------------+--------------------+--------------------+--------------------+
|         created_at|            hashtags|                text|           hash_text|
+-------------------+--------------------+--------------------+--------------------+
|2014-10-01 07:27:52|[[WrappedArray(64...|RT @2pmalways: RT...|   [trueswag, jun_k]|
|2014-10-02 18:53:08|[[WrappedArray(40...|Que esa frase me ...|[guerreroencanalf...|
|2014-10-03 05:01:07|[[WrappedArray(75...|New Warfare Has E...|[dalycity, bayvie...|
|2014-10-04 00:15:00|[[WrappedArray(41...|RT @dafnehurley: ...|[rt, fav, dinámicas]|
|2014-10-04 05:18:56|[[WrappedArray(99...|M16 Sentinel Mech...|[vape, vaporizer,...|
|2014-10-04 21:39:47|[[WrappedArray(62...|秋のお出かけなら【登別温泉】へ行き...|    [北海道, 旅行, 秋, 紅葉]|
|2014-10-05 17:20:46|[[WrappedArray(57...|RT @realmadrid: ....|[realmadridathlet...|
|2014-10-06 10:47:52|[[WrappedArray(91...|RT @Asr3Follow01:...|[ريتويت, تابعني_ا...|
|2014-10-07 14:32:00|[[WrappedArray(24...|@Marcelapq10 than...|  

Convert the tweet text to lowercase, split it into a word array and add this into a new column called test. Then check if any of the elements of keys is in this list and add this information in the column intext.

In [29]:
# key = 'rt'
keys = ['@2pmalways:', 'rt']

hashtag_df = hashtag_df.withColumn('test', key_udf_lower('text'))
hashtag_df = hashtag_df.withColumn('test', key_udf_split('test'))
hashtag_df = hashtag_df.withColumn('intext', key_udf_key('test'))
hashtag_df.show(10)

+-------------------+--------------------+--------------------+--------------------+--------------------+------+
|         created_at|            hashtags|                text|           hash_text|                test|intext|
+-------------------+--------------------+--------------------+--------------------+--------------------+------+
|2014-10-01 07:27:52|[[WrappedArray(64...|RT @2pmalways: RT...|   [trueswag, jun_k]|[rt, @2pmalways:,...|  true|
|2014-10-02 18:53:08|[[WrappedArray(40...|Que esa frase me ...|[guerreroencanalf...|[que, esa, frase,...| false|
|2014-10-03 05:01:07|[[WrappedArray(75...|New Warfare Has E...|[dalycity, bayvie...|[new, warfare, ha...| false|
|2014-10-04 00:15:00|[[WrappedArray(41...|RT @dafnehurley: ...|[rt, fav, dinámicas]|[rt, @dafnehurley...|  true|
|2014-10-04 05:18:56|[[WrappedArray(99...|M16 Sentinel Mech...|[vape, vaporizer,...|[m16, sentinel, m...| false|
|2014-10-04 21:39:47|[[WrappedArray(62...|秋のお出かけなら【登別温泉】へ行き...|    [北海道, 旅行, 秋, 紅葉]|[秋のお出かけなら【登別

Compute the day ID of the tweet and add it to the column new_date

In [31]:
hashtag_df = hashtag_df.withColumn('new_date', new_date('created_at'))

In [32]:
hashtag_df.show(10)

+-------------------+--------------------+--------------------+--------------------+--------------------+------+--------+
|         created_at|            hashtags|                text|           hash_text|                test|intext|new_date|
+-------------------+--------------------+--------------------+--------------------+--------------------+------+--------+
|2014-10-01 07:27:52|[[WrappedArray(64...|RT @2pmalways: RT...|   [trueswag, jun_k]|[rt, @2pmalways:,...|  true| 2014274|
|2014-10-02 18:53:08|[[WrappedArray(40...|Que esa frase me ...|[guerreroencanalf...|[que, esa, frase,...| false| 2014275|
|2014-10-03 05:01:07|[[WrappedArray(75...|New Warfare Has E...|[dalycity, bayvie...|[new, warfare, ha...| false| 2014276|
|2014-10-04 00:15:00|[[WrappedArray(41...|RT @dafnehurley: ...|[rt, fav, dinámicas]|[rt, @dafnehurley...|  true| 2014277|
|2014-10-04 05:18:56|[[WrappedArray(99...|M16 Sentinel Mech...|[vape, vaporizer,...|[m16, sentinel, m...| false| 2014277|
|2014-10-04 21:39:47|[[W

Questions:
* What does the text method do?
* Why does the new_date function work even though it is not an UDF?